In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
running_order.head()

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,1,6.0,fiona bunn,1,Fiona Bunn,1.0,2.110,0.172,6.743671,1.863840,2.033659,0.084909,0.100000,2.110,0.100000
1,1,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,2,5.7,joana wälti,2,Joana Wälti,2.0,2.102,0.124,6.882701,1.809328,2.053610,0.122141,0.122141,2.102,0.122141
2,2,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,3,7.3,anna kindlundh,3,Anna Kindlundh,6.0,2.077,0.120,6.107582,1.726928,2.056589,0.164830,0.164830,2.077,0.120000
3,3,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,4,7.9,jeanette jönsson hellstadius,4,Jeanette Jönsson Hellstadius,3.0,2.041,0.134,7.474857,1.845341,2.089668,0.122164,0.122164,2.041,0.134000
4,4,30,Suunta Jyväskylä 1,Suunta Jyväskylä,FIN,1,6.0,veera klemettinen,1,Veera Klemettinen,6.0,2.064,0.106,7.198831,1.873431,2.050207,0.088388,0.100000,2.064,0.106000


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,8,Södertälje Nykvarn Orientering 1,1,6.0,Fiona Bunn,2.110,0.100000
1,8,Södertälje Nykvarn Orientering 1,2,5.7,Joana Wälti,2.102,0.122141
2,8,Södertälje Nykvarn Orientering 1,3,7.3,Anna Kindlundh,2.077,0.120000
3,8,Södertälje Nykvarn Orientering 1,4,7.9,Jeanette Jönsson Hellstadius,2.041,0.134000
4,30,Suunta Jyväskylä 1,1,6.0,Veera Klemettinen,2.064,0.106000


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                    \
leg                                     1                                 2   
team_id                                                                       
8        Södertälje Nykvarn Orientering 1  Södertälje Nykvarn Orientering 1   
30                     Suunta Jyväskylä 1                Suunta Jyväskylä 1   
37                    Farum-Tisvilde OK 1               Farum-Tisvilde OK 1   
41              Lahden Suunnistajat -37 1         Lahden Suunnistajat -37 1   
46                             OK Orion 1                        OK Orion 1   

                                                                             \
leg                                     3                                 4   
team_id                                                                       
8        Södertälje Nykvarn Orientering 1  Södertälje Nykvarn Orientering 1   
30                     Suunta Jyväskylä 1                Suunta Jyväskylä 1   
37                    Farum-Tisvilde OK 1               Farum-Tisvilde OK 1   
41              Lahden Suunnistajat -37 1         Lahden Suunnistajat -37 1   
46                             OK Orion 1                        OK Orion 1   

        dist                              name                    \
leg        1    2    3    4                  1                 2   
team_id                                                            
8        6.0  5.7  7.3  7.9         Fiona Bunn       Joana Wälti   
30       6.0  5.7  7.3  7.9  Veera Klemettinen  Elisa Viholainen   
37       6.0  5.7  7.3  7.9       Agnes Kracht       Iben Valery   
41       6.0  5.7  7.3  7.9     Laura Ramstein  Lotta Kirvesmies   
46       6.0  5.7  7.3  7.9       Ebba Berglin          Ida Öbro   

                                                           log_mean            \
leg                       3                             4         1         2   
team_id                                                                         
8            Anna Kindlundh  Jeanette Jönsson Hellstadius  2.110000  2.102000   
30       Barbora Chaloupska             Kaisa Klemettinen  2.064000  2.061000   
37          Mathilde Kracht                   Masha Semak  2.131000  1.983129   
41        Anna-Kaisa Anjala                  Minna Kauppi  1.973000  1.968000   
46        Hanna Wiesniewska               Tilda Johansson  2.025348  1.978000   

                           log_std                             
leg             3      4         1         2         3      4  
team_id                                                        
8        2.077000  2.041  0.100000  0.122141  0.120000  0.134  
30       1.984000  2.050  0.106000  0.122000  0.140614  0.138  
37       2.073000  2.108  0.106113  0.123215  0.156757  0.197  
41       2.024000  2.025  0.116000  0.097000  0.147421  0.109  
46       2.041707  1.927  0.118492  0.145000  0.156277  0.131

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(689, 20)

(678, 20)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
8,40.606282,82.768234,136.250459,191.910558,60.155090,112.194182,176.878260,244.021167,49.455802,96.379817,...,46.165612,46.555633,60.155090,58.942184,73.689069,79.189623,49.455802,46.607085,58.204061,60.850469
30,38.366731,78.885623,126.677248,182.821204,57.789469,108.114819,168.359636,236.459888,47.197124,92.304191,...,40.384487,47.041270,57.789469,57.092111,69.945225,80.247960,47.197124,44.640049,53.140493,61.422984
37,41.084695,78.967269,129.217150,185.333949,62.164985,108.088589,176.240548,255.171073,50.596094,92.245325,...,42.529121,44.349199,62.164985,52.702260,78.668812,95.261647,50.596094,41.453853,57.972604,65.100520
41,34.385685,72.255772,121.746045,177.906171,53.943842,97.476300,161.414617,225.293871,43.078510,84.040127,...,41.443828,48.638945,53.943842,49.354838,73.148929,74.250879,43.078510,40.767228,55.285975,59.928204
46,36.076603,72.667710,122.805062,173.510140,57.233362,104.337554,169.493183,227.196613,45.450239,87.139565,...,41.377933,42.077222,57.233362,54.231002,76.219687,69.905321,45.450239,41.298633,56.311686,54.168753


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,name_3,name_4,log_mean_1,log_mean_2,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4
team_id,,,,,,,,,,,,,,,,,,,,
8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,6.0,5.7,7.3,7.9,Fiona Bunn,Joana Wälti,Anna Kindlundh,Jeanette Jönsson Hellstadius,2.11,2.10,2.08,2.04,0.10,0.12,0.12,0.13
30,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,6.0,5.7,7.3,7.9,Veera Klemettinen,Elisa Viholainen,Barbora Chaloupska,Kaisa Klemettinen,2.06,2.06,1.98,2.05,0.11,0.12,0.14,0.14
37,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,6.0,5.7,7.3,7.9,Agnes Kracht,Iben Valery,Mathilde Kracht,Masha Semak,2.13,1.98,2.07,2.11,0.11,0.12,0.16,0.20
41,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,6.0,5.7,7.3,7.9,Laura Ramstein,Lotta Kirvesmies,Anna-Kaisa Anjala,Minna Kauppi,1.97,1.97,2.02,2.02,0.12,0.10,0.15,0.11
46,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion 1,6.0,5.7,7.3,7.9,Ebba Berglin,Ida Öbro,Hanna Wiesniewska,Tilda Johansson,2.03,1.98,2.04,1.93,0.12,0.14,0.16,0.13


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,6.0,5.7,7.3,7.9,Fiona Bunn,Joana Wälti,...,46.165612,46.555633,60.155090,58.942184,73.689069,79.189623,49.455802,46.607085,58.204061,60.850469
30,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,6.0,5.7,7.3,7.9,Veera Klemettinen,Elisa Viholainen,...,40.384487,47.041270,57.789469,57.092111,69.945225,80.247960,47.197124,44.640049,53.140493,61.422984
37,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,Farum-Tisvilde OK 1,6.0,5.7,7.3,7.9,Agnes Kracht,Iben Valery,...,42.529121,44.349199,62.164985,52.702260,78.668812,95.261647,50.596094,41.453853,57.972604,65.100520
41,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,6.0,5.7,7.3,7.9,Laura Ramstein,Lotta Kirvesmies,...,41.443828,48.638945,53.943842,49.354838,73.148929,74.250879,43.078510,40.767228,55.285975,59.928204
46,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion 1,6.0,5.7,7.3,7.9,Ebba Berglin,Ida Öbro,...,41.377933,42.077222,57.233362,54.231002,76.219687,69.905321,45.450239,41.298633,56.311686,54.168753


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist  \
team_id leg                                           
8       1    Södertälje Nykvarn Orientering 1   6.0   
        2    Södertälje Nykvarn Orientering 1   5.7   
        3    Södertälje Nykvarn Orientering 1   7.3   
        4    Södertälje Nykvarn Orientering 1   7.9   
30      1                  Suunta Jyväskylä 1   6.0   
        2                  Suunta Jyväskylä 1   5.7   
        3                  Suunta Jyväskylä 1   7.3   

                                     name  log_mean   log_std  fin_start95  \
team_id leg                                                                  
8       1                      Fiona Bunn     2.110  0.100000    40.606282   
        2                     Joana Wälti     2.102  0.122141    82.768234   
        3                  Anna Kindlundh     2.077  0.120000   136.250459   
        4    Jeanette Jönsson Hellstadius     2.041  0.134000   191.910558   
30      1               Veera Klemettinen     2.064  0.106000    38.366731   
        2                Elisa Viholainen     2.061  0.122000    78.885623   
        3              Barbora Chaloupska     1.984  0.140614   126.677248   

              fin_end95  fin_median    fin_mean  fin_sum_log_mean  \
team_id leg                                                         
8       1     60.155090   49.455802   49.684962          3.900706   
        2    112.194182   96.379817   96.671141          4.568277   
        3    176.878260  155.135956  155.370302          5.043588   
        4    244.021167  216.329116  216.787415          5.377044   
30      1     57.789469   47.197124   47.472196          3.854609   
        2    108.114819   92.304191   92.530480          4.524327   
        3    168.359636  145.803531  146.209258          4.982413   

             fin_sum_log_std  mass_start  dark_during_leg  personal_start95  \
team_id leg                                                                   
8       1           0.099975         0.0              0.0         40.606282   
        2           0.077937         0.0              0.0         36.631769   
        3           0.066677         0.0              0.0         46.165612   
        4           0.061193         0.0              0.0         46.555633   
30      1           0.105289         0.0              0.0         38.366731   
        2           0.080142         0.0              0.0         35.331726   
        3           0.072462         0.0              0.0         40.384487   

             personal_end95  personal_median  
team_id leg                                   
8       1         60.155090        49.455802  
        2         58.942184        46.607085  
        3         73.689069        58.204061  
        4         79.189623        60.850469  
30      1         57.789469        47.197124  
        2         57.092111        44.640049  
        3         69.945225        53.140493

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [15]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist  \
team_id leg                                           
8       1    Södertälje Nykvarn Orientering 1   6.0   
        2    Södertälje Nykvarn Orientering 1   5.7   
        3    Södertälje Nykvarn Orientering 1   7.3   
        4    Södertälje Nykvarn Orientering 1   7.9   
30      1                  Suunta Jyväskylä 1   6.0   

                                     name  log_mean   log_std  fin_start95  \
team_id leg                                                                  
8       1                      Fiona Bunn     2.110  0.100000    40.606282   
        2                     Joana Wälti     2.102  0.122141    82.768234   
        3                  Anna Kindlundh     2.077  0.120000   136.250459   
        4    Jeanette Jönsson Hellstadius     2.041  0.134000   191.910558   
30      1               Veera Klemettinen     2.064  0.106000    38.366731   

              fin_end95  fin_median    fin_mean  fin_sum_log_mean  \
team_id leg                                                         
8       1     60.155090   49.455802   49.684962          3.900706   
        2    112.194182   96.379817   96.671141          4.568277   
        3    176.878260  155.135956  155.370302          5.043588   
        4    244.021167  216.329116  216.787415          5.377044   
30      1     57.789469   47.197124   47.472196          3.854609   

             fin_sum_log_std  mass_start  dark_during_leg  personal_start95  \
team_id leg                                                                   
8       1           0.099975         0.0              0.0         40.606282   
        2           0.077937         0.0              0.0         36.631769   
        3           0.066677         0.0              0.0         46.165612   
        4           0.061193         0.0              0.0         46.555633   
30      1           0.105289         0.0              0.0         38.366731   

             personal_end95  personal_median               fin_time_median  \
team_id leg                                                                  
8       1         60.155090        49.455802 2019-06-15 14:49:27.348091841   
        2         58.942184        46.607085 2019-06-15 15:36:22.789007664   
        3         73.689069        58.204061 2019-06-15 16:35:08.157362461   
        4         79.189623        60.850469 2019-06-15 17:36:19.746979713   
30      1         57.789469        47.197124 2019-06-15 14:47:11.827451944   

                         fin_time_start95                fin_time_end95  \
team_id leg                                                               
8       1   2019-06-15 14:40:36.376890898 2019-06-15 15:00:09.305415869   
        2   2019-06-15 15:22:46.094033480 2019-06-15 15:52:11.650909901   
        3   2019-06-15 16:16:15.027536154 2019-06-15 16:56:52.695612907   
        4   2019-06-15 17:11:54.633464098 2019-06-15 18:04:01.270018339   
30      1   2019-06-15 14:38:22.003861666 2019-06-15 14:57:47.368115425   

             personal_estimate_interval  
team_id leg                              
8       1                     19.548809  
        2                     22.310416  
        3                     27.523456  
        4                     32.633990  
30      1                     19.422738

In [16]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

#for_print[for_print['team'].str.contains("Stadin")]
for_print = for_print[for_print['team'].str.contains("Reak")]

for_print = for_print[[
 'leg',
 'name', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,


In [17]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'ind_log_mean', 
 'ind_log_std',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,ind_log_mean,ind_log_std,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
348,341,1,Rasti-Jyry 1,Jola Maisala,2.197000,0.117000,3.985774,0.115867,2019-06-15 11:42:53.693395376+00:00,2019-06-15 12:07:36.227687120+00:00,2019-06-15 11:53:47.970592499+00:00,0.0000,0.0
349,341,2,Rasti-Jyry 1,Kaija Rantoja,2.364000,0.140000,4.744604,0.092648,2019-06-15 12:35:54.151877880+00:00,2019-06-15 13:17:38.092402697+00:00,2019-06-15 12:55:06.976068497+00:00,0.0000,0.0
350,341,3,Rasti-Jyry 1,Jenni Castren,2.395000,0.152000,5.276373,0.082834,2019-06-15 13:46:58.617831469+00:00,2019-06-15 14:49:56.312352896+00:00,2019-06-15 14:15:39.341670752+00:00,0.0000,0.0
351,341,4,Rasti-Jyry 1,Inka Lemmetyinen,2.558000,0.178309,5.700297,0.082871,2019-06-15 15:14:06.042733192+00:00,2019-06-15 16:52:23.822825193+00:00,2019-06-15 15:58:52.735176325+00:00,0.0000,0.0
616,528,1,Rasti-Jyry 2,Päivi Taponen,2.418000,0.163000,4.209725,0.164517,2019-06-15 11:48:38.351341963+00:00,2019-06-15 12:32:43.832803249+00:00,2019-06-15 12:07:21.705415726+00:00,0.0000,0.0
617,528,2,Rasti-Jyry 2,Miia Hovi,2.229000,0.141000,4.796828,0.111572,2019-06-15 12:37:25.945376396+00:00,2019-06-15 13:30:34.301416636+00:00,2019-06-15 13:01:13.180093050+00:00,0.0000,0.0
618,528,3,Rasti-Jyry 2,Mervi Luurila,2.459000,0.191317,5.335727,0.102839,2019-06-15 13:50:12.192228556+00:00,2019-06-15 15:14:37.545429707+00:00,2019-06-15 14:27:21.459930420+00:00,0.0000,0.0
619,528,4,Rasti-Jyry 2,Tiina Isoniemi,2.371000,0.116000,5.679178,0.080246,2019-06-15 15:10:08.829004049+00:00,2019-06-15 16:43:42.049451113+00:00,2019-06-15 15:52:27.550415039+00:00,0.0063,0.0
920,740,1,Rasti-Jyry 4,Mila Viksilä,2.581289,0.144432,4.374595,0.144139,2019-06-15 11:59:53.096326828+00:00,2019-06-15 12:45:29.333898306+00:00,2019-06-15 12:19:24.026548862+00:00,0.0000,0.0
921,740,2,Rasti-Jyry 4,Tuula Ohramaa,2.617000,0.174000,5.066449,0.112399,2019-06-15 13:06:55.610321283+00:00,2019-06-15 14:17:45.321402788+00:00,2019-06-15 13:38:26.413200855+00:00,0.0000,0.0


In [18]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")